In [ ]:

import json 
with open("padel-data-labels/labels/2022_BCN_finalF_1_ball.json", "r") as f:
            data = json.load(f)
##Getting the keys of the json file 
for key in data : 
        print(key)      

licenses
info
categories
images
annotations


In [4]:
##in  our first case we will only focus on the Ball data  
print(data["categories"])

[{'id': 1, 'name': 'Ball', 'supercategory': ''}, {'id': 2, 'name': 'Wall', 'supercategory': ''}, {'id': 3, 'name': 'shot-event', 'supercategory': ''}, {'id': 4, 'name': 'serve-event', 'supercategory': ''}, {'id': 5, 'name': 'smash-event', 'supercategory': ''}, {'id': 6, 'name': 'forehand-event', 'supercategory': ''}, {'id': 7, 'name': 'backhand-event', 'supercategory': ''}, {'id': 8, 'name': 'other-shot-event', 'supercategory': ''}, {'id': 9, 'name': 'drop-shot-event', 'supercategory': ''}]


In [5]:
for image in data["images"][:10]:
    print(image)

{'id': 1, 'width': 1920, 'height': 1080, 'file_name': 'frame_000000.PNG', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
{'id': 2, 'width': 1920, 'height': 1080, 'file_name': 'frame_000001.PNG', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
{'id': 3, 'width': 1920, 'height': 1080, 'file_name': 'frame_000002.PNG', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
{'id': 4, 'width': 1920, 'height': 1080, 'file_name': 'frame_000003.PNG', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
{'id': 5, 'width': 1920, 'height': 1080, 'file_name': 'frame_000004.PNG', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
{'id': 6, 'width': 1920, 'height': 1080, 'file_name': 'frame_000005.PNG', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
{'id': 7, 'width': 1920, 'height': 1080, 'file_name': 'frame_000006.PNG', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
{'id':

In [6]:
for annotation in data["annotations"][:10]:
    print(annotation)

{'id': 1, 'image_id': 1, 'category_id': 1, 'segmentation': [], 'area': 61.12679999999947, 'bbox': [864.45, 338.02, 7.98, 7.66], 'iscrowd': 0, 'attributes': {'occluded': False, 'rotation': 0.0}}
{'id': 2, 'image_id': 2, 'category_id': 1, 'segmentation': [], 'area': 66.16719999999985, 'bbox': [867.7, 354.29, 8.24, 8.03], 'iscrowd': 0, 'attributes': {'occluded': False, 'rotation': 0.0}}
{'id': 3, 'image_id': 3, 'category_id': 1, 'segmentation': [], 'area': 66.16720000000032, 'bbox': [867.7, 354.28, 8.24, 8.03], 'iscrowd': 0, 'attributes': {'occluded': False, 'rotation': 0.0}}
{'id': 4, 'image_id': 4, 'category_id': 1, 'segmentation': [], 'area': 62.408400000000086, 'bbox': [868.66, 348.1, 7.94, 7.86], 'iscrowd': 0, 'attributes': {'occluded': False, 'rotation': 0.0}}
{'id': 5, 'image_id': 5, 'category_id': 1, 'segmentation': [], 'area': 62.328999999999716, 'bbox': [869.19, 337.76, 7.94, 7.85], 'iscrowd': 0, 'attributes': {'occluded': False, 'rotation': 0.0}}
{'id': 6, 'image_id': 6, 'categ

In [10]:
#create a new dict with the necessary for our model 
data_ball= {}
for image in data["images"]:
    data_ball[image["id"]] = {"id": image["id"],"image_name": image["file_name"] }
for annotation in data["annotations"][:10]:    
    data_ball[annotation["image_id"]]["bbox"] = annotation["bbox"]
    data_ball[annotation["image_id"]]["area"] = annotation["area"]

for ball in list(data_ball)[:5]:
    print(data_ball[ball])    

{'id': 1, 'image_name': 'frame_000000.PNG', 'bbox': [864.45, 338.02, 7.98, 7.66], 'area': 61.12679999999947}
{'id': 2, 'image_name': 'frame_000001.PNG', 'bbox': [867.7, 354.29, 8.24, 8.03], 'area': 66.16719999999985}
{'id': 3, 'image_name': 'frame_000002.PNG', 'bbox': [867.7, 354.28, 8.24, 8.03], 'area': 66.16720000000032}
{'id': 4, 'image_name': 'frame_000003.PNG', 'bbox': [868.66, 348.1, 7.94, 7.86], 'area': 62.408400000000086}
{'id': 5, 'image_name': 'frame_000004.PNG', 'bbox': [869.19, 337.76, 7.94, 7.85], 'area': 62.328999999999716}
